In [3]:
import acquire as a
import pandas as pd

In [ ]:
#process only one repo
test= a.process_repo('dsfsi/covid19za')

In [ ]:
#take a look
test

In [4]:
scrape_data = a.scrape_github_data()

In [5]:
df = pd.DataFrame(scrape_data)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   repo             100 non-null    object
 1   language         89 non-null     object
 2   readme_contents  100 non-null    object
dtypes: object(3)
memory usage: 2.5+ KB


# Prepare

In [6]:
import prepare

IndentationError: unindent does not match any outer indentation level (prepare.py, line 34)

In [7]:
prep_df= prepare.prep_data(df, 'readme_contents', extra_words=[], exclude_words=[])

NameError: name 'prepare' is not defined

In [ ]:
df = df.rename({'readme_contents':'original'},axis=1)
df['clean'] = df.original.apply(prepare.basic_clean)
df['lemmatized'] = df.clean.apply(prepare.lemmatize)
df['stopwords_removed'] = df.lemmatized.apply(prepare.remove_stopwords)
df.head()

# Explore

In [ ]:
#see the count for each language
df.language.value_counts()

#python appears to be the most used language

In [ ]:
#take a look at all python 
df[df.language == 'Python'].clean

In [ ]:
#python clean
python_words = clean(' '.join(df[df.label == 'Python'].clean))
python_words